# **<span style='color:#F1A424'>Innavation Workshop - 2022. Quick success</span>**
#### **<span style='color:#F1A424'>Practical part of Quick Success, Skoltech, Moscow-2022</span>**

**Instructors:** Elizveta Kiseleva, Mikhail Gasanov, Anna Petrovskaia

To prepare this seminar the following resources were used:

[kaggle](https://www.kaggle.com/code/shtrausslearning/pytorch-cnn-binary-image-classification)


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import copy
import os
import torch
from PIL import Image
from PIL import Image, ImageDraw
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
from torchvision import utils
%matplotlib inline

In [ ]:
!pip install torchsummary

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Dataset

## Load dataset

- Load the dataset information file; train_labels.csv, it contains a reference to an image ID (id) & its classification allocation (label)

In [ ]:
labels_df = pd.read_csv('c)
labels_df.head()

In [ ]:
labels_df.shape

## Check for duplicate entries

- Check if the dataset contains any duplicates, if there is we should drop them, which we have none

In [ ]:
# No duplicate ids found
labels_df[labels_df.duplicated(keep=False)]

## Target feature class balance
- Let's check the number of object in each class

In [ ]:
labels_df['label'].value_counts()

## Dataset preview

- Let's also visualise the dataset images

In [ ]:
imgpath ="/mnt/bulky/apetrovskaya/workshop/train/" # training data is stored in this folder

In [ ]:
train_img_names_raw = os.listdir(imgpath)
train_img_names  = [f.split('.')[0] for f in train_img_names_raw]

In [ ]:
labels_df = labels_df[labels_df.id.isin(train_img_names)]

In [ ]:
malignant = labels_df.loc[labels_df['label']==1]['id'].values    # get the ids of malignant cases
normal = labels_df.loc[labels_df['label']==0]['id'].values       # get the ids of the normal cases

In [ ]:
nrows,ncols=6,15
fig,ax = plt.subplots(nrows,ncols,figsize=(15,6))
plt.subplots_adjust(wspace=0, hspace=0) 
for i,j in enumerate(malignant[:nrows*ncols]):
    fname = os.path.join(imgpath ,j +'.tif')
    img = Image.open(fname)
    idcol = ImageDraw.Draw(img)
    idcol.rectangle(((0,0),(95,95)),outline='red')
    plt.subplot(nrows, ncols, i+1) 
    plt.imshow(np.array(img))
    plt.axis('off')

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)
plt.subplots_adjust(wspace=0, hspace=0)

nrows,ncols=6,15
for i,j in enumerate(normal[:nrows*ncols]):
    fname = os.path.join(imgpath ,j +'.tif')
    img = Image.open(fname)
    idcol = ImageDraw.Draw(img)
    idcol.rectangle(((0,0),(95,95)),outline='green')
    plt.subplot(nrows, ncols, i+1) 
    plt.imshow(np.array(img))
    plt.axis('off')

# Data Preparation

- Let's create a custom <code>Dataset</code> class by subclassing the <code>Pytorch Dataset</code> class:
    - We need just two essential fuctions <code>__len__</code> & <code>__getitem__</code> in our custom class

In [ ]:
torch.manual_seed(0) # fix random seed

class pytorch_data(Dataset):
    
    def __init__(self,data_dir,transform,data_type="train"):      
    
        # Get Image File Names
        cdm_data=os.path.join(data_dir,data_type)  # directory of files
        
        file_names = os.listdir(cdm_data) # get list of images in that directory  
        idx_choose = np.random.choice(np.arange(len(file_names)), 
                                      4000,
                                      replace=False).tolist()
        file_names_sample = [file_names[x] for x in idx_choose]
        self.full_filenames = [os.path.join(cdm_data, f) for f in file_names_sample]   # get the full path to images
        
        # Get Labels
        labels_data=os.path.join(data_dir,"train_labels.csv") 
        labels_df=pd.read_csv(labels_data)
        labels_df.set_index("id", inplace=True) # set data frame index to id
        self.labels = [labels_df.loc[filename[:-4]].values[0] for filename in file_names_sample]  # obtained labels from df
        self.transform = transform
      
    def __len__(self):
        return len(self.full_filenames) # size of dataset
      
    def __getitem__(self, idx):
        # open image, apply transforms and return with label
        image = Image.open(self.full_filenames[idx])  # Open Image with PIL
        image = self.transform(image) # Apply Specific Transformation to Image
        return image, self.labels[idx]


In [ ]:
# define transformation that converts a PIL image into PyTorch tensors
import torchvision.transforms as transforms
data_transformer = transforms.Compose([transforms.ToTensor(),
                                       transforms.Resize((46,46))])

In [ ]:
# Define an object of the custom dataset for the train folder.
data_dir = '/mnt/bulky/apetrovskaya/workshop/'
img_dataset = pytorch_data(data_dir, data_transformer, "train") # Histopathalogic images

In [ ]:
# load an example tensor
img,label=img_dataset[10]
print(img.shape,torch.min(img),torch.max(img))

## Splitting the Dataset

- Among the training set, we need to evaluate the model on validation datasets to track the model's performance during training.
- Let's use 20% of img_dataset for validation & use the rest as the training set, so we have a 80/20 split

In [ ]:
len_img=len(img_dataset)
len_train=int(0.8*len_img)
len_val=len_img-len_train

# Split Pytorch tensor
train_ts,val_ts=random_split(img_dataset,
                             [len_train,len_val]) # random split 80/20

print("train dataset size:", len(train_ts))
print("validation dataset size:", len(val_ts))

In [ ]:
# getting the torch tensor image & target variable
ii=-1
for x,y in train_ts:
    print(x.shape,y)
    ii+=1
    if(ii>5):
        break

In [ ]:
import plotly.express as px

def plot_img(x,y,title=None):

    npimg = x.numpy() # convert tensor to numpy array
    npimg_tr=np.transpose(npimg, (1,2,0)) # Convert to H*W*C shape
    fig = px.imshow(npimg_tr)
    fig.update_layout(template='plotly_white')
    fig.update_layout(title=title,height=300,margin={'l':10,'r':20,'b':10})
    fig.show()

## Training subset examples

- Some examples from our training data subset, with corresponding labels.

In [ ]:
import plotly.express as px

# Create grid of sample images 
grid_size=30
rnd_inds=np.random.randint(0,len(train_ts),grid_size)
print("image indices:",rnd_inds)

x_grid_train=[train_ts[i][0] for i in rnd_inds]
y_grid_train=[train_ts[i][1] for i in rnd_inds]

x_grid_train=utils.make_grid(x_grid_train, nrow=10, padding=2)
print(x_grid_train.shape)
    
plot_img(x_grid_train,y_grid_train,'Training Subset Examples')

## Validation subset examples

- Some examples from the validation subset, with corresponding labels.

In [ ]:
grid_size=30
rnd_inds=np.random.randint(0,len(val_ts),grid_size)
print("image indices:",rnd_inds)
x_grid_val=[val_ts[i][0] for i in range(grid_size)]
y_grid_val=[val_ts[i][1] for i in range(grid_size)]

x_grid_val=utils.make_grid(x_grid_val, nrow=10, padding=2)
print(x_grid_val.shape)

plot_img(x_grid_val,y_grid_val,'Validation Dataset Preview')

# Transforming the Dataset

#### **<span style='color:#F1A424'>IMAGE AUGMENTATIONS</span>**

- Among with pretrained models, image __transformation__ and __image augmentation__ are generally considered to be an essential parts of constructing deep learning models.
- Using image transformations, we can expand our dataset or resize and normalise it to achieve better model performance.
- Typical transformations include __horizontal__,__vertical flipping__, __rotation__, __resizing__.
- We can use various image transformations for our binary classification model without making label changes; we can flip/rotate a image but it will remain the same class.
- We can use the torchvision module to perform image transformations during the training process.

#### **<span style='color:#F1A424'>TRAINING DATA AUGMENTATIONS</span>**
- transforms.RandomHorizontalFlip(p=0.5): Flips the image horizontally with the probability of 0.5
- transforms.RandomVerticalFlip(p=0.5) : Flips the image vertically  " 
- transforms.RandomRotation(45) : Rotates the images in the range of (-45,45) degrees.
- transforms.RandomResizedCrop(96,scale=(0.8,1.0),ratio=(1.0,1.0)) : Randomly square crops the image in the range of [72,96], followed by a resize to 96x96, which is the original pixel size of our image data.
- transforms.ToTensor() : Converts to Tensor & Normalises as shown above already.

In [ ]:
# Define the following transformations for the training dataset
tr_transf = transforms.Compose([
#     transforms.Resize((40,40)),
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomVerticalFlip(p=0.5),  
    transforms.RandomRotation(45),         
#     transforms.RandomResizedCrop(50,scale=(0.8,1.0),ratio=(1.0,1.0)),
    transforms.ToTensor()])

In [ ]:
# For the validation dataset, we don't need any augmentation; simply convert images into tensors
val_transf = transforms.Compose([
    transforms.ToTensor()])

# After defining the transformations, overwrite the transform functions of train_ts, val_ts
train_ts.transform=tr_transf
val_ts.transform=val_transf

# Creating Dataloaders

- Ready to create a PyTorch Dataloader. If we don't use __Dataloaders__, we have to write code to loop over datasets & extract a data batch; automated.
- We need to define a __batch_size__ : The number of images extracted from the dataset each iteration

In [ ]:
from torch.utils.data import DataLoader

# Training DataLoader
train_dl = DataLoader(train_ts,
                      batch_size=32, 
                      shuffle=True)

# Validation DataLoader
val_dl = DataLoader(val_ts,
                    batch_size=32,
                    shuffle=False)

In [ ]:
for x,y in train_dl:
    print(x.shape,y)
    break

# Binary Classifier CNN Model

- Model is comprised of
  - **<span style='color:#F1A424'>four CNN</span>** **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Conv2D</mark>** layers with a **<span style='color:#F1A424'>pooling layer</span>** **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">max_pool2D</mark>** added between each layer 
  - Two **<span style='color:#F1A424'>fully connected</span>** layers **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">fc</mark>**, with a **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">dropout</mark>** layer between the two layers
  - **<span style='color:#F1A424'>log_softmax</span>** is used as the activation function for the final layer of the **<span style='color:#F1A424'>binary classifier</span>**
- PyTorch allows us to create a custom class with <code>nn.Module</code>


In [ ]:
# Useful Function to calculate the output size of a CNN layer
# before making it an input into the linear layer

def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Neural Network
class Network(nn.Module):
    
    # Network Initialisation
    def __init__(self, params):
        
        super(Network, self).__init__()
    
        Cin,Hin,Win=params["shape_in"]
        init_f=params["initial_filters"] 
        num_fc1=params["num_fc1"]  
        num_classes=params["num_classes"] 
        self.dropout_rate=params["dropout_rate"] 
        
        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv4)
        
        # compute the flatten size
        self.num_flatten=h*w*8*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self,X):
        
        # Convolution & Pool Layers
        X = F.relu(self.conv1(X)); X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X)); X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X));X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X));X = F.max_pool2d(X, 2, 2)

        X = X.view(-1, self.num_flatten)
        
        X = F.relu(self.fc1(X))
        X=F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)

In [ ]:
# Neural Network Predefined Parameters
params_model={
        "shape_in": (3,46,46), 
        "initial_filters": 8,    
        "num_fc1": 100,
        "dropout_rate": 0.25,
        "num_classes": 2}

# Create instantiation of Network class
cnn_model = Network(params_model)

# define computation hardware approach (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = cnn_model.to(device)

# Defining a Loss Function

- Loss Functions are one of the key pieces of an effective deep learning solution.
- Pytorch uses <code>loss functions</code> to determine how it will update the network to reach the desired solution.
- The standard loss function for classification tasks is __cross entropy loss__ or __logloss__
- When defining a loss function, we need to consider, the number of model outputs and their activation functions.
- For binary classification tasks, we can choose one or two outputs.
- It is recommended to use __log_softmax__ as it is easier to expand to multiclass classification; PyTorch combines the log and softmax operations into one function, due to numerical stability and speed.

In [ ]:
loss_func = nn.NLLLoss(reduction="sum")

#  Defining an Optimiser

- Training the network involves passing data through the network:
    - Using the **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">loss function</mark>** to **<span style='color:#F1A424'>determine the difference between the prediction & true value</span>**
    - Which is then followed by using of that info to **<span style='color:#F1A424'>update the weights</span>** of the network 
    - In an attempt to **<span style='color:#F1A424'>make the loss function return as small of a loss as possible, performing updates on the neural network</span>**, an **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">optimiser</mark>** is used
- The <code>torch.optim</code> contains implementations of common optimisers
- The **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">optimiser</mark>** will **<span style='color:#F1A424'>hold the current state and will update the parameters based on the computed gradients</mark>**
- For binary classification taskss, __SGD__, __Adam__ Optimisers are commonly used, let's use the latter here.

In [ ]:
from torch import optim
opt = optim.Adam(cnn_model.parameters(), lr=3e-4)
lr_scheduler = ReduceLROnPlateau(opt, mode='min',factor=0.5, patience=20,verbose=1)

#  Training the Model

## Helper functions

- The main training loop function <code>train_val</code> will utiliser three functions:
    - <code>get_lr</code> : get the learning rate as it is adjusted 
    - <code>loss_batch</code> : get the loss value for the particular batch
    - <code>loss_epoch</code> : get the entire loss for an epoch iteration

In [ ]:
''' Helper Functions'''

# Function to get the learning rate
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

# Function to compute the loss value per batch of data
def loss_batch(loss_func, output, target, opt=None):
    
    loss = loss_func(output, target) # get loss
    pred = output.argmax(dim=1, keepdim=True) # Get Output Class
    metric_b=pred.eq(target.view_as(pred)).sum().item() # get performance metric
    
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()

    return loss.item(), metric_b

# Compute the loss value & performance metric for the entire dataset (epoch)
def loss_epoch(model,loss_func,dataset_dl,check=False,opt=None):
    
    run_loss=0.0 
    t_metric=0.0
    len_data=len(dataset_dl.dataset)

    # internal loop over dataset
    for xb, yb in dataset_dl:
        # move batch to device
        xb=xb.to(device)
        yb=yb.to(device)
        output=model(xb) # get model output
        loss_b,metric_b=loss_batch(loss_func, output, yb, opt) # get loss per batch
        run_loss+=loss_b        # update running loss

        if metric_b is not None: # update running metric
            t_metric+=metric_b

        # break the loop in case of sanity check
        if check is True:
            break
    
    loss=run_loss/float(len_data)  # average loss value
    metric=t_metric/float(len_data) # average metric value
    
    return loss, metric

## Main training function

In [ ]:
def train_val(model, params,verbose=False):
    
    # Get the parameters
    epochs=params["epochs"]
    loss_func=params["f_loss"]
    opt=params["optimiser"]
    train_dl=params["train"]
    val_dl=params["val"]
    check=params["check"]
    lr_scheduler=params["lr_change"]
    weight_path=params["weight_path"]
    
    loss_history={"train": [],"val": []} # history of loss values in each epoch
    metric_history={"train": [],"val": []} # histroy of metric values in each epoch
    best_model_wts = copy.deepcopy(model.state_dict()) # a deep copy of weights for the best performing model
    best_loss=float('inf') # initialize best loss to a large value
    
    # main loop
    for epoch in range(epochs):
        
        ''' Get the Learning Rate '''
        current_lr=get_lr(opt)
        if(verbose):
            print('Epoch {}/{}, current lr={}'.format(epoch, epochs - 1, current_lr))
        
        ''' Train the Model on the Training Set '''
        model.train()
        train_loss, train_metric=loss_epoch(model,loss_func,train_dl,check,opt)

        ''' Collect loss and metric for training dataset ''' 
        loss_history["train"].append(train_loss)
        metric_history["train"].append(train_metric)
        
        ''' Evaluate model on validation dataset '''
        model.eval()
        with torch.no_grad():
            val_loss, val_metric=loss_epoch(model,loss_func,val_dl,check)
        
        # store best model
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            
            # store weights into a local file
            torch.save(model.state_dict(), weight_path)
            if(verbose):
                print("Copied best model weights!")
        
        # collect loss and metric for validation dataset
        loss_history["val"].append(val_loss)
        metric_history["val"].append(val_metric)
        
        # learning rate schedule
        lr_scheduler.step(val_loss)
        if current_lr != get_lr(opt):
            if(verbose):
                print("Loading best model weights!")
            model.load_state_dict(best_model_wts) 

        if(verbose):
            print(f"train loss: {train_loss:.6f}, dev loss: {val_loss:.6f}, accuracy: {100*val_metric:.2f}")
            print("-"*10) 

    # load best model weights
    model.load_state_dict(best_model_wts)
        
    return model, loss_history, metric_history

In [ ]:
params_train={
 "train": train_dl,"val": val_dl,
 "epochs": 50,
 "optimiser": optim.Adam(cnn_model.parameters(),
                         lr=3e-4),
 "lr_change": ReduceLROnPlateau(opt,
                                mode='min',
                                factor=0.5,
                                patience=20,
                                verbose=0),
 "f_loss": nn.NLLLoss(reduction="sum"),
 "weight_path": "weights.pt",
 "check": False, 
}

''' Actual Train / Evaluation of CNN Model '''
# train and validate the model
cnn_model,loss_hist,metric_hist=train_val(cnn_model,params_train)

In [ ]:
# Train-Validation Progress
epochs=params_train["epochs"]

fig = make_subplots(rows=1, cols=2,subplot_titles=['lost_hist','metric_hist'])
fig.add_trace(go.Scatter(x=[*range(1,epochs+1)], y=loss_hist["train"],name='loss_hist["train"]'),row=1, col=1)
fig.add_trace(go.Scatter(x=[*range(1,epochs+1)], y=loss_hist["val"],name='loss_hist["val"]'),row=1, col=1)
fig.add_trace(go.Scatter(x=[*range(1,epochs+1)], y=metric_hist["train"],name='metric_hist["train"]'),row=1, col=2)
fig.add_trace(go.Scatter(x=[*range(1,epochs+1)], y=metric_hist["val"],name='metric_hist["val"]'),row=1, col=2)
fig.update_layout(template='plotly_white')
fig.update_layout(margin={"r":0,"t":60,"l":0,"b":0},height=300)
fig.show()

## Obtain label for one image

In [ ]:
image_path=imgpath+train_img_names_raw[0]
image = plt.imread(image_path)
plt.imshow(image);

In [ ]:
my_transforms = transforms.Compose([transforms.ToTensor(),transforms.Resize((46,46))])
image_tensor = my_transforms(image).unsqueeze(0) 
output=cnn_model(image_tensor)
pred = output.argmax(dim=1, keepdim=True)
print(pred)